In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
sns.set_style('darkgrid')

In [ ]:
df= pd.read_csv('../input/house-rent-prices-of-metropolitan-cities-in-india/Hyderabad_rent.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df=df.drop_duplicates().reset_index(drop=True)

In [ ]:
df.describe(include='all')

In [ ]:
df['bathroom'].replace(np.nan,'0',inplace=True)

In [ ]:
for i in range(0,len(df['bathroom'])):
    df['bathroom'].iloc[i]=df['bathroom'].iloc[i].split()[0]

In [ ]:
df['bathroom'].replace(np.nan,'0',inplace=True)

In [ ]:
df['bathroom'].unique()

In [ ]:
# Droping data that has no information about bathroom

df=df.drop(list(df[df['bathroom']=='East'].index))
df=df.drop(list(df[df['bathroom']=='West'].index))
df=df.drop(list(df[df['bathroom']=='North'].index))
df=df.drop(list(df[df['bathroom']=='South'].index))
df=df.drop(list(df[df['bathroom']=='Grfloor'].index))
df=df.drop(list(df[df['bathroom']=='NorthWest'].index))

In [ ]:
df['bathroom']=df['bathroom'].astype(int) #converting bathroom column into integer column

In [ ]:
df.describe(include='all')

In [ ]:
### There are few houses with zero bathrooms which is not possible,hence,replacing the number of bathrooms based on the property type

In [ ]:
df[df['bathroom']==0]

In [ ]:
df['property_type'].unique()

In [ ]:
#### considering property type Studio Apartment

df[df['property_type']=='Studio Apartment'].describe()

In [ ]:
###### It is clear that most of the studio apartments have a single bathroom, hence, replacing 0 with 1

for i in range(0,len(df['property_type'])):
    if df['property_type'].iloc[i]=='Studio Apartment':
        df['bathroom'].iloc[i]=1

In [ ]:
#### considering property type Villa

df[df['property_type']=='Villa'].describe()

In [ ]:
###### It is clear that 50% of the Villas have a 3 bathrooms, hence, replacing 0 with 3

for i in range(0,len(df['property_type'])):
    if df['property_type'].iloc[i]=='Villa':
        df['bathroom'].iloc[i]=3

In [ ]:
#### considering property type Independent House

df[df['property_type']=='Independent House'].describe()

In [ ]:
#### The mean is close to 2 and median is 2, replacing 0 with 2

for i in range(0,len(df['property_type'])):
    if df['property_type'].iloc[i]=='Independent House':
        df['bathroom'].iloc[i]=2

In [ ]:
#### considering property type Independent Floor

df[df['property_type']=='Independent Floor'].describe()

In [ ]:
#### The mean is close to 2 and median is 2, replacing 0 with 2

for i in range(0,len(df['property_type'])):
    if df['property_type'].iloc[i]=='Independent Floor':
        df['bathroom'].iloc[i]=2

In [ ]:
#### considering property type Apartment

df[df['property_type']=='Apartment'].describe()

for i in range(0,len(df['property_type'])):
    if (df['property_type'].iloc[i]=='Apartment') & (df['bedroom'].iloc[i]>1):
        df['bathroom'].iloc[i]=2
    elif (df['property_type'].iloc[i]=='Apartment') & (df['bedroom'].iloc[i]==1):
        df['bathroom'].iloc[i]=1

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df.describe(include='all')

In [ ]:
df['price']= [i.replace(',','') for i in df['price']]

In [ ]:
df['price']=df['price'].astype(float)

In [ ]:
for i in range(0,len(df)):
    
    if df['price'].iloc[i]<10:
        df['price'].iloc[i]=df['price'].iloc[i]*100000

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x=df.index,y=df['price'],hue=df['property_type'])

#df.to_excel('Hyderabad_rentals.xlsx')

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x=df['area'],y=df['price'],hue=df['bedroom'],palette='viridis')

df.columns

In [ ]:
df['price/area']= df['price']/df['area']

In [ ]:
len(df.locality.unique())

In [ ]:
locality_stats=df.groupby('locality')['locality'].count().sort_values(ascending=False)

In [ ]:
len(locality_stats[locality_stats<=10])

In [ ]:
locality_less_than_10=locality_stats[locality_stats<=10]

In [ ]:
df.locality= df.locality.apply(lambda x: 'other' if x in locality_less_than_10 else x)

In [ ]:
df= df.drop(['seller_type','layout_type'],axis=1)

In [ ]:
for i in ['property_type','locality','furnish_type']:
    df[i]=df[i].astype('category')

In [ ]:
df['property_type_cat']=df['property_type'].cat.codes

In [ ]:
df['locality_cat']=df['locality'].cat.codes

In [ ]:
df['furnish_type_cat']=df['furnish_type'].cat.codes

In [ ]:
df[df['price/area']>100]

In [ ]:
df['price/area'].idxmax()

In [ ]:
df=df.drop([2690,2241,3838,4532])

In [ ]:
df['price/area'].describe()

In [ ]:
plt.figure(figsize=(20,10))
sns.histplot(df['price'],kde=True)

In [ ]:
sns.boxplot(df['price'])

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x=df.index,y=df['price'])

In [ ]:
df[df['price']>100000]

In [ ]:
df=df[~(df['price']>60000)]

In [ ]:
plt.figure(figsize=(20,10))
sns.histplot(df['price'],kde=True)

In [ ]:
df.shape

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
df.describe()

In [ ]:
for i in ['property_type_cat','locality_cat','furnish_type_cat']:
    df[i]=df[i].astype('category')

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X=df.drop(['price','property_type','locality','furnish_type'],axis=1)

In [ ]:
ss=StandardScaler()

In [ ]:
y=df['price']

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_scaled_train=pd.DataFrame(ss.fit_transform(X_train),columns=X.columns)

In [ ]:
X_scaled_test=pd.DataFrame(ss.transform(X_test),columns=X.columns)

In [ ]:
# LightGBM parameters
lgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.01, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     hp.choice('n_estimators', np.arange(10,1000,50)),
}
lgb_fit_params = {
    'eval_metric': 'l2',
    'early_stopping_rounds': 10,
    'verbose': False
}
lgb_para = dict()
lgb_para['reg_params'] = lgb_reg_params
lgb_para['fit_params'] = lgb_fit_params
lgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))

In [ ]:
class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def lgb_reg(self, para):
        reg = lgb.LGBMRegressor(**para['reg_params'])
        return self.train_reg(reg, para)
    
    def train_reg(self, reg, para):
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        return {'loss': loss, 'status': STATUS_OK}

In [ ]:
obj = HPOpt(X_scaled_train, X_scaled_test, y_train, y_test)

In [ ]:
lgb_opt = obj.process(fn_name='lgb_reg', space=lgb_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

In [ ]:
lgb_opt

In [ ]:
np.arange(0.3, 0.8, 0.1)[4]

In [ ]:
np.arange(0.01, 0.31, 0.05)[1]

In [ ]:
np.arange(5, 16, 1, dtype=int)[9]

In [ ]:
np.arange(1, 8, 1, dtype=int)[3]

In [ ]:
np.arange(10,1000,50)[15]

In [ ]:
lgb_r=lgb.LGBMRegressor(n_estimators=760,colsample_bytree=0.7,learning_rate=0.06,max_depth=14,min_child_weight=4,subsample=0.9416535612780146)

In [ ]:
lgb_r.fit(X_scaled_train,y_train)

In [ ]:
y_hat=lgb_r.predict(X_scaled_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,y_hat))

In [ ]:
mean_absolute_error(y_test,y_hat)

In [ ]:
r2_score(y_test,y_hat)

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x=range(0,len(y_test)),y=y_test)
sns.scatterplot(x=range(0,len(y_test)),y=y_hat)


In [ ]:
feature_importances = lgb_r.feature_importances_

In [ ]:
importance=pd.DataFrame(columns=['Feature','Importance'])

In [ ]:
importance['Feature']=X_train.columns

In [ ]:
importance['Importance']=feature_importances

In [ ]:
importance= importance.sort_values(by='Importance',ascending=False)

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x=importance['Importance'],y=importance['Feature'])

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(x=range(0,len(y_test)),y=(y_test-y_hat))
sns.lineplot(x=range(0,len(y_test)),y=mean_absolute_error(y_test,y_hat),label='MAE',color='red')
sns.lineplot(x=range(0,len(y_test)),y=np.sqrt(mean_squared_error(y_test,y_hat)),label='MSE',color='orange')